# Creacion de subgrupos pequeños de tif con todos mis hdf

### Ver los archivos de mi directorio

In [5]:
import os

# Ver directorio actual
print(os.getcwd())

# Cambiar de directorio
os.chdir(r"C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4")

# Verificar que cambiaste
print("Nuevo directorio:", os.getcwd())

# Ver contenido del directorio
print("Archivos en el directorio:")
for archivo in os.listdir():
    print(archivo)

c:\Users\usuario\AppData\Local\Programs\Microsoft VS Code
Nuevo directorio: C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4
Archivos en el directorio:
.netrc
123.txt
231.txt
cache
convert_hdf_to_tif.py
download_modis_lads.py
hw3_241878.ipynb
modis_data
peru_protected_areas.gpkg
requirements.txt
shape_file
temp_batch_0.tif
temp_batch_1.tif
temp_batch_10.tif
temp_batch_11.tif
temp_batch_12.tif
temp_batch_13.tif
temp_batch_14.tif
temp_batch_15.tif
temp_batch_16.tif
temp_batch_17.tif
temp_batch_18.tif
temp_batch_19.tif
temp_batch_2.tif
temp_batch_20.tif
temp_batch_21.tif
temp_batch_22.tif
temp_batch_23.tif
temp_batch_24.tif
temp_batch_25.tif
temp_batch_26.tif
temp_batch_27.tif
temp_batch_28.tif
temp_batch_29.tif
temp_batch_3.tif
temp_batch_30.tif
temp_batch_31.tif
temp_batch_32.tif
temp_batch_33.tif
temp_batch_34.tif
temp_batch_35.tif
temp_batch_36.tif
temp_batch_4.tif
temp_batch_5.tif
temp_batch_6.tif
temp_batch_7.tif
temp_batch_8.tif
temp_batch_9.t

In [8]:
import os

# Ver directorio actual
print(os.getcwd())

# Cambiar de directorio
os.chdir(r"C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4\modis_data\tifs")

# Verificar que cambiaste
print("Nuevo directorio:", os.getcwd())

# Ver contenido del directorio
print("Archivos en el directorio:")
for archivo in os.listdir():
    print(archivo)

C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4
Nuevo directorio: C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4\modis_data\tifs
Archivos en el directorio:
MOD44B.A2024065.h00v08.061.2025133154709_TreeCover.tif
MOD44B.A2024065.h00v09.061.2025133154708_TreeCover.tif
MOD44B.A2024065.h00v10.061.2025133154818_TreeCover.tif
MOD44B.A2024065.h01v07.061.2025133154328_TreeCover.tif
MOD44B.A2024065.h01v08.061.2025133155408_TreeCover.tif
MOD44B.A2024065.h01v09.061.2025133154428_TreeCover.tif
MOD44B.A2024065.h01v10.061.2025133154427_TreeCover.tif
MOD44B.A2024065.h01v11.061.2025133154533_TreeCover.tif
MOD44B.A2024065.h02v06.061.2025133155049_TreeCover.tif
MOD44B.A2024065.h02v08.061.2025133155047_TreeCover.tif
MOD44B.A2024065.h02v09.061.2025133154511_TreeCover.tif
MOD44B.A2024065.h02v10.061.2025133155112_TreeCover.tif
MOD44B.A2024065.h02v11.061.2025133154448_TreeCover.tif
MOD44B.A2024065.h03v06.061.2025133155419_TreeCov

### 🌲 3.2 Descarga y procesamiento del VCF (MOD44B v061) – Año 2024

En esta sección, automatizamos la descarga y procesamiento de los datos MODIS VCF (Vegetation Continuous Fields, producto `MOD44B v061`) correspondientes al año 2024. Este producto proporciona estimaciones de cobertura arbórea a nivel global con una resolución de aproximadamente 250 metros.

#### 🔹 Pasos realizados:

3. **Creación de mosaico nacional**  
   - Se unifican los archivos `.tif` usando procesamiento por lotes para optimizar el uso de memoria, debido al gran tamaño total (291 archivos).
   - El mosaico final conserva la resolución original y valores de cobertura arbórea sin alteraciones.

#### ⚙️ Configuración técnica:
- ✅ Directorio de entrada: `modis_data/tifs/`  
- ✅ Archivo de salida: `vcf_2024_peru.tif`  
- ✅ Resolución: ~231.66 m/píxel  
- ✅ CRS original: proyección sinusoidal MODIS (sin reproyección en este paso)  
- ✅ Procesamiento: dividido en 37 lotes

#### ⚠️ Nota:
Durante la creación del mosaico final, se intentó unir todos los archivos en una sola operación. Sin embargo, el proceso falló por un error de asignación de memoria (más de **46 GB** necesarios para combinar todos los píxeles). Este problema puede resolverse:
- Dividiendo la combinación final en subgrupos más pequeños
- Usando técnicas como `gdalbuildvrt` + `gdal_translate`
- Ejecutando en un entorno con más memoria disponible

Esta etapa es clave, ya que el mosaico resultante se usará para calcular estadísticas de cobertura forestal en zonas cercanas a áreas protegidas.


In [10]:
import os
import glob
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling
from rasterio.windows import Window
import numpy as np
from collections import defaultdict
import math

# Directorios
TIFS_DIR = r"C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4\modis_data\tifs"
OUTPUT_DIR = r"C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4"
MOSAIC_TEMP = os.path.join(OUTPUT_DIR, "vcf_2024_peru_temp.tif")
FINAL_OUTPUT = os.path.join(OUTPUT_DIR, "vcf_2024_peru.tif")

# Configuración de memoria
MAX_CHUNK_SIZE = 1000  # Reducir si sigues teniendo problemas de memoria
BATCH_SIZE = 20  # Procesar archivos en lotes más pequeños

def get_mosaic_bounds_and_resolution(tif_files):
    """Calcular bounds y resolución del mosaico sin cargar todos los datos"""
    print("📐 Calculando bounds del mosaico...")
    
    all_bounds = []
    resolutions = []
    crs_list = []
    
    for tif_file in tif_files:
        with rasterio.open(tif_file) as src:
            all_bounds.append(src.bounds)
            resolutions.append(src.res)
            crs_list.append(src.crs)
    
    # Verificar que todos tienen el mismo CRS
    if not all(crs == crs_list[0] for crs in crs_list):
        print("⚠️ Advertencia: Los archivos tienen diferentes CRS")
    
    # Calcular bounds del mosaico
    left = min(bounds.left for bounds in all_bounds)
    bottom = min(bounds.bottom for bounds in all_bounds)
    right = max(bounds.right for bounds in all_bounds)
    top = max(bounds.top for bounds in all_bounds)
    
    # Usar la resolución más común
    res_x = np.median([res[0] for res in resolutions])
    res_y = np.median([res[1] for res in resolutions])
    
    return (left, bottom, right, top), (res_x, res_y), crs_list[0]

def create_mosaic_by_batches():
    """Crear mosaico procesando archivos en lotes para reducir uso de memoria"""
    print("🔍 Buscando archivos TIFF...")
    
    # Buscar todos los archivos TreeCover.tif
    tif_pattern = os.path.join(TIFS_DIR, "*TreeCover.tif")
    tif_files = glob.glob(tif_pattern)
    
    if not tif_files:
        print(f"❌ No se encontraron archivos TreeCover.tif en {TIFS_DIR}")
        return False
    
    print(f"✅ Encontrados {len(tif_files)} archivos TIFF")
    
    try:
        # Obtener información del mosaico
        mosaic_bounds, resolution, src_crs = get_mosaic_bounds_and_resolution(tif_files)
        
        # Calcular dimensiones del mosaico
        width = int((mosaic_bounds[2] - mosaic_bounds[0]) / resolution[0])
        height = int((mosaic_bounds[3] - mosaic_bounds[1]) / abs(resolution[1]))
        
        print(f"📏 Dimensiones del mosaico: {width} x {height}")
        print(f"📐 Resolución: {resolution}")
        
        # Crear transformación
        from rasterio.transform import from_bounds
        transform = from_bounds(*mosaic_bounds, width, height)
        
        # Configurar metadatos de salida
        profile = {
            'driver': 'GTiff',
            'dtype': 'float32',
            'nodata': None,
            'width': width,
            'height': height,
            'count': 1,
            'crs': src_crs,
            'transform': transform,
            'compress': 'lzw',
            'tiled': True,
            'blockxsize': 512,
            'blockysize': 512
        }
        
        print("🔄 Creando mosaico por chunks...")
        
        # Crear archivo de salida
        with rasterio.open(MOSAIC_TEMP, 'w', **profile) as dst:
            # Procesar por chunks para manejar memoria
            chunk_height = min(MAX_CHUNK_SIZE, height)
            chunk_width = min(MAX_CHUNK_SIZE, width)
            
            total_chunks = math.ceil(height / chunk_height) * math.ceil(width / chunk_width)
            processed_chunks = 0
            
            for row in range(0, height, chunk_height):
                for col in range(0, width, chunk_width):
                    # Definir ventana del chunk
                    chunk_h = min(chunk_height, height - row)
                    chunk_w = min(chunk_width, width - col)
                    
                    window = Window(col, row, chunk_w, chunk_h)
                    
                    # Inicializar array del chunk
                    chunk_data = np.full((chunk_h, chunk_w), np.nan, dtype=np.float32)
                    chunk_mask = np.zeros((chunk_h, chunk_w), dtype=bool)
                    
                    # Procesar archivos en lotes para este chunk
                    for batch_start in range(0, len(tif_files), BATCH_SIZE):
                        batch_files = tif_files[batch_start:batch_start + BATCH_SIZE]
                        
                        for tif_file in batch_files:
                            try:
                                with rasterio.open(tif_file) as src:
                                    # Leer datos del chunk desde este archivo
                                    try:
                                        data = src.read(1, window=window, boundless=True, fill_value=np.nan)
                                        valid_mask = ~np.isnan(data) & (data != src.nodata if src.nodata is not None else True)
                                        
                                        # Actualizar chunk donde hay datos válidos
                                        update_mask = valid_mask & ~chunk_mask
                                        chunk_data[update_mask] = data[update_mask]
                                        chunk_mask |= valid_mask
                                        
                                    except Exception as e:
                                        # Si hay error leyendo este chunk de este archivo, continuar
                                        continue
                                        
                            except Exception as e:
                                print(f"⚠️ Error procesando {tif_file}: {e}")
                                continue
                    
                    # Escribir chunk al archivo de salida
                    dst.write(chunk_data, 1, window=window)
                    
                    processed_chunks += 1
                    if processed_chunks % 10 == 0:
                        progress = (processed_chunks / total_chunks) * 100
                        print(f"   Progreso: {progress:.1f}% ({processed_chunks}/{total_chunks} chunks)")
        
        print("✅ Mosaico creado exitosamente")
        return True
        
    except Exception as e:
        print(f"❌ Error creando mosaico: {e}")
        return False

def create_mosaic_simple():
    """Versión alternativa usando merge en lotes más pequeños - PRESERVA RESOLUCIÓN ORIGINAL PARA RDD"""
    print("🔍 Buscando archivos TIFF...")
    
    tif_pattern = os.path.join(TIFS_DIR, "*TreeCover.tif")
    tif_files = glob.glob(tif_pattern)
    
    if not tif_files:
        print(f"❌ No se encontraron archivos TreeCover.tif en {TIFS_DIR}")
        return False
    
    print(f"✅ Encontrados {len(tif_files)} archivos TIFF")
    
    # Verificar resolución original para análisis RDD
    with rasterio.open(tif_files[0]) as sample:
        original_res = sample.res
        original_crs = sample.crs
        print(f"📐 Resolución original: {original_res} (preservada para análisis píxel-nivel)")
        print(f"📍 CRS original: {original_crs}")
    
    # Dividir archivos en lotes
    batch_size = 8  # Reducir a 8 para ser más conservador con memoria
    batches = [tif_files[i:i + batch_size] for i in range(0, len(tif_files), batch_size)]
    
    print(f"📦 Procesando en {len(batches)} lotes de máximo {batch_size} archivos")
    
    try:
        temp_mosaics = []
        
        # Procesar cada lote
        for i, batch in enumerate(batches):
            print(f"🔄 Procesando lote {i+1}/{len(batches)} ({len(batch)} archivos)...")
            
            src_files_to_mosaic = []
            try:
                # Abrir archivos del lote
                for tif_file in batch:
                    src = rasterio.open(tif_file)
                    src_files_to_mosaic.append(src)
                
                # Crear mosaico del lote
                mosaic, out_trans = merge(src_files_to_mosaic)
                
                # Guardar mosaico temporal del lote
                temp_mosaic_path = os.path.join(OUTPUT_DIR, f"temp_batch_{i}.tif")
                temp_mosaics.append(temp_mosaic_path)
                
                # Configurar metadatos manteniendo dtype original para precisión en RDD
                out_meta = src_files_to_mosaic[0].meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": mosaic.shape[1],
                    "width": mosaic.shape[2],
                    "transform": out_trans,
                    "compress": "lzw",
                    "tiled": True,
                    # Preservar dtype original para análisis estadístico preciso
                    "dtype": src_files_to_mosaic[0].dtypes[0],
                    # Preservar nodata original
                    "nodata": src_files_to_mosaic[0].nodata
                })
                
                with rasterio.open(temp_mosaic_path, "w", **out_meta) as dest:
                    dest.write(mosaic)
                
                # Cerrar archivos fuente
                for src in src_files_to_mosaic:
                    src.close()
                
                # Limpiar memoria
                del mosaic
                
            except Exception as e:
                print(f"❌ Error en lote {i+1}: {e}")
                for src in src_files_to_mosaic:
                    try:
                        src.close()
                    except:
                        pass
                return False
        
        # Ahora combinar todos los mosaicos temporales
        print("🔄 Combinando mosaicos temporales...")
        temp_src_files = []
        try:
            for temp_path in temp_mosaics:
                temp_src_files.append(rasterio.open(temp_path))
            
            final_mosaic, final_trans = merge(temp_src_files)
            
            # Configurar metadatos preservando información original para RDD
            out_meta = temp_src_files[0].meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": final_mosaic.shape[1],
                "width": final_mosaic.shape[2],
                "transform": final_trans,
                "compress": "lzw",
                "tiled": True,
                # Mantener dtype y nodata originales para análisis preciso
                "dtype": temp_src_files[0].dtypes[0],
                "nodata": temp_src_files[0].nodata
            })
            
            with rasterio.open(MOSAIC_TEMP, "w", **out_meta) as dest:
                dest.write(final_mosaic)
            
            # Cerrar archivos temporales
            for src in temp_src_files:
                src.close()
            
            # Eliminar archivos temporales
            for temp_path in temp_mosaics:
                if os.path.exists(temp_path):
                    os.remove(temp_path)
            
            print("✅ Mosaico final creado exitosamente")
            return True
            
        except Exception as e:
            print(f"❌ Error combinando mosaicos: {e}")
            for src in temp_src_files:
                try:
                    src.close()
                except:
                    pass
            return False
        
    except Exception as e:
        print(f"❌ Error general: {e}")
        return False

def reproject_to_epsg4326():
    """Reproyectar mosaico a EPSG:4326 PRESERVANDO RESOLUCIÓN PARA ANÁLISIS RDD"""
    print("🌐 Reproyectando a EPSG:4326...")
    
    if not os.path.exists(MOSAIC_TEMP):
        print(f"❌ No se encuentra el mosaico temporal: {MOSAIC_TEMP}")
        return False
    
    try:
        with rasterio.open(MOSAIC_TEMP) as src:
            # Definir CRS de destino
            dst_crs = 'EPSG:4326'
            
            # Calcular transformación MANTENIENDO resolución apropiada para RDD
            # Usamos resolución conservativa para preservar detalles
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds,
                resolution=None  # Usar resolución automática para preservar detalle
            )
            
            # Configurar metadatos PRESERVANDO información crítica para RDD
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height,
                'compress': 'lzw',
                'tiled': True,
                'blockxsize': 512,
                'blockysize': 512,
                # CRÍTICO: Mantener dtype y nodata originales para estadísticas zonales precisas
                'dtype': src.dtypes[0],
                'nodata': src.nodata
            })
            
            print(f"📏 Dimensiones originales: {src.width} x {src.height}")
            print(f"📏 Dimensiones reproyectadas: {width} x {height}")
            print(f"📍 CRS original: {src.crs}")
            print(f"📍 CRS destino: {dst_crs}")
            print(f"🎯 Dtype preservado: {src.dtypes[0]} (importante para RDD)")
            print(f"🎯 NoData preservado: {src.nodata}")
            
            # Crear archivo reproyectado CON RESAMPLING APROPIADO PARA VCF
            with rasterio.open(FINAL_OUTPUT, 'w', **kwargs) as dst:
                # Reproyectar banda por banda para manejar memoria
                for i in range(1, src.count + 1):
                    print(f"🔄 Reproyectando banda {i}/{src.count}...")
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        # CRÍTICO: Usar nearest neighbor para preservar valores originales VCF
                        resampling=Resampling.nearest,
                        num_threads=2  # Limitar threads para reducir uso de memoria
                    )
        
        print(f"✅ Archivo reproyectado guardado: {FINAL_OUTPUT}")
        
        # Eliminar archivo temporal
        if os.path.exists(MOSAIC_TEMP):
            os.remove(MOSAIC_TEMP)
            print("🗑️ Archivo temporal eliminado")
        
        return True
        
    except Exception as e:
        print(f"❌ Error reproyectando: {e}")
        return False

def verify_output():
    """Verificar el archivo de salida - VALIDACIÓN ESPECIAL PARA RDD"""
    if not os.path.exists(FINAL_OUTPUT):
        print("❌ El archivo final no existe")
        return
    
    try:
        with rasterio.open(FINAL_OUTPUT) as src:
            print(f"\n📊 Información del archivo final (para análisis RDD):")
            print(f"   Archivo: {FINAL_OUTPUT}")
            print(f"   Dimensiones: {src.width} x {src.height}")
            print(f"   Bandas: {src.count}")
            print(f"   CRS: {src.crs} ✅ (EPSG:4326 requerido)")
            print(f"   Extent: {src.bounds}")
            print(f"   Resolución: {src.res}")
            print(f"   Dtype: {src.dtypes[0]} (importante para estadísticas)")
            print(f"   NoData: {src.nodata}")
            
            # Validación crítica para RDD - verificar rango de valores VCF
            print("\n🎯 Validación para análisis VCF:")
            sample_data = src.read(1, window=((0, min(1000, src.height)), (0, min(1000, src.width))))
            
            if src.nodata is not None:
                valid_data = sample_data[sample_data != src.nodata]
            else:
                valid_data = sample_data[~np.isnan(sample_data)]
            
            if len(valid_data) > 0:
                print(f"   📈 Rango VCF - Min: {valid_data.min():.1f}%, Max: {valid_data.max():.1f}%, Media: {valid_data.mean():.1f}%")
                
                # Verificar que los valores están en rango esperado para VCF (0-100)
                if valid_data.min() >= 0 and valid_data.max() <= 100:
                    print("   ✅ Valores VCF en rango válido (0-100%)")
                else:
                    print("   ⚠️ ADVERTENCIA: Algunos valores VCF fuera del rango 0-100%")
                
                # Estadísticas adicionales para RDD
                percentiles = np.percentile(valid_data, [25, 50, 75, 90, 95])
                print(f"   📊 Percentiles: P25={percentiles[0]:.1f}, P50={percentiles[1]:.1f}, P75={percentiles[2]:.1f}, P90={percentiles[3]:.1f}, P95={percentiles[4]:.1f}")
                
                # Verificar cobertura (% de píxeles con datos)
                coverage = (len(valid_data) / sample_data.size) * 100
                print(f"   📊 Cobertura de datos: {coverage:.1f}% (muestra)")
                
            else:
                print("   ❌ CRÍTICO: No se encontraron valores válidos - revisar nodata")
            
            # Verificar tamaño del archivo
            file_size_mb = os.path.getsize(FINAL_OUTPUT) / (1024 * 1024)
            print(f"   💾 Tamaño: {file_size_mb:.1f} MB")
            
            # Validación geográfica para Perú
            bounds = src.bounds
            # Bounds aproximados de Perú: [-81.3, -18.4, -68.7, -0.0]
            if (bounds.left < -82 and bounds.right > -68 and 
                bounds.bottom < -19 and bounds.top > -1):
                print("   🇵🇪 ✅ Cobertura geográfica incluye Perú")
            else:
                print("   🇵🇪 ⚠️ Verificar cobertura geográfica de Perú")
            
    except Exception as e:
        print(f"❌ Error verificando archivo: {e}")

def main():
    """Función principal - OPTIMIZADO PARA TAREA RDD"""
    print("🚀 Iniciando creación de mosaico VCF 2024 para Perú")
    print("🎯 CONFIGURADO ESPECÍFICAMENTE PARA ANÁLISIS RDD")
    print(f"📂 Directorio de entrada: {TIFS_DIR}")
    print(f"📂 Archivo de salida: {FINAL_OUTPUT}")
    
    # Verificar directorios
    if not os.path.exists(TIFS_DIR):
        print(f"❌ Error: No existe el directorio {TIFS_DIR}")
        return
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print("\n🤔 Método optimizado para memoria:")
    print("   Usando procesamiento por lotes (recomendado para RDD)")
    print("   ✅ Preserva resolución original")
    print("   ✅ Mantiene valores VCF precisos")
    print("   ✅ Conserva dtype original")
    
    # Paso 1: Crear mosaico con configuración para RDD
    print("\n📦 Paso 1/3: Creando mosaico...")
    success = create_mosaic_simple()
    
    if not success:
        print("❌ Error creando mosaico")
        return
    
    # Paso 2: Reproyectar a EPSG:4326 (requerido por tarea)
    print("\n🌐 Paso 2/3: Reproyectando a EPSG:4326...")
    if not reproject_to_epsg4326():
        print("❌ Error reproyectando")
        return
    
    # Paso 3: Verificar resultado con validaciones específicas para RDD
    print("\n✅ Paso 3/3: Verificando resultado...")
    verify_output()
    
    print(f"\n🎉 ¡Mosaico VCF 2024 creado exitosamente para análisis RDD!")
    print(f"📁 Archivo final: {FINAL_OUTPUT}")
    print(f"\n🚀 Listo para continuar con:")
    print(f"   📊 Sección 3.3: Estadísticas zonales con buffers")
    print(f"   📈 Sección 3.5: Análisis RDD a nivel píxel")

if __name__ == "__main__":
    main()

🚀 Iniciando creación de mosaico VCF 2024 para Perú
🎯 CONFIGURADO ESPECÍFICAMENTE PARA ANÁLISIS RDD
📂 Directorio de entrada: C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4\modis_data\tifs
📂 Archivo de salida: C:\Users\usuario\Documents\GitHub\Spatial_Analysis_of_Peru-s_Protected_Areas\241878_hw4\vcf_2024_peru.tif

🤔 Método optimizado para memoria:
   Usando procesamiento por lotes (recomendado para RDD)
   ✅ Preserva resolución original
   ✅ Mantiene valores VCF precisos
   ✅ Conserva dtype original

📦 Paso 1/3: Creando mosaico...
🔍 Buscando archivos TIFF...
✅ Encontrados 291 archivos TIFF
📐 Resolución original: (231.65635826395825, 231.65635826395834) (preservada para análisis píxel-nivel)
📍 CRS original: PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not specified (based on custom spheroid)",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]]